In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
db = sql.connect('rpg_db.sqlite3')

In [3]:
pd.read_sql_query("pragma table_info(sqlite_master)", db)

,cid,name,type,notnull,dflt_value,pk
0,0,type,text,0,None,0
1,1,name,text,0,None,0
2,2,tbl_name,text,0,None,0
3,3,rootpage,int,0,None,0
4,4,sql,text,0,None,0


In [4]:
pd.read_sql_query("select name from sqlite_master where type = 'table'", db)

,name
0,django_migrations
1,sqlite_sequence
2,auth_group
3,auth_group_permissions
4,auth_user_groups
5,auth_user_user_permissions
6,django_admin_log
7,armory_item
8,armory_weapon
9,django_content_type


#### How many total Characters are there?

In [5]:
pd.read_sql_query("select count(*) from charactercreator_character", db)

,count(*)
0,302


#### How many of each specific subclass?

In [17]:
pd.read_sql_query("""
    select 
        (select count(*) from charactercreator_cleric) as cleric,
        (select count(*) from charactercreator_fighter) as fighter,
        (select count(*) from charactercreator_mage) as mage,
        (select count(*) from charactercreator_thief) as thief
    """, 
    db
)

,cleric,fighter,mage,thief
0,75,68,108,51


#### How many total Items?

In [25]:
pd.read_sql_query("select count(item_id) from armory_item", db)

,count(item_id)
0,174


#### How many of the Items are weapons? How many are not?

In [28]:
pd.read_sql_query(
    """
    select count(*) 
    from armory_weapon 
    inner join armory_item 
    on armory_item.item_id = armory_weapon.item_ptr_id
    """, 
    db
)

,count(*)
0,37


#### How many Items does each character have? (Return first 20 rows)

In [39]:
pd.read_sql_query(
    """
    select count(item_id) from charactercreator_character_inventory group by character_id limit 20
    """,
    db
)

,count(item_id)
0,3
1,3
2,2
3,4
4,4
5,1
6,5
7,3
8,4
9,4


#### How many Weapons does each character have? (Return first 20 rows)

In [41]:
pd.read_sql_query(
    """
    select count(inventory.item_id) 
    from charactercreator_character_inventory as inventory
    inner join armory_weapon as weapons
    on inventory.item_id = weapons.item_ptr_id
    group by inventory.character_id limit 20
    """,
    db
)

,count(inventory.item_id)
0,2
1,1
2,1
3,1
4,1
5,1
6,1
7,3
8,2
9,1


#### On average, how many Items does each Character have?

In [51]:
pd.read_sql_query(
    """
    select
        avg(item_counts)
    from (select count(inventory.item_id) as item_counts
          from charactercreator_character_inventory as inventory)
    """,
    db
)

,avg(item_counts)
0,898.0


#### On average, how many Weapons does each character have?

In [52]:
pd.read_sql_query(
    """
    select 
        avg(item_counts)
    from (select count(inventory.item_id) as item_counts
          from charactercreator_character_inventory as inventory
          inner join armory_weapon as weapons
          on inventory.item_id = weapons.item_ptr_id
          group by inventory.character_id)
    """,
    db
)

,avg(item_counts)
0,1.309677
